In [1]:
import import_ipynb
from etherscan_scrapper_v0 import *
import pandas as pd

importing Jupyter notebook from etherscan_scrapper_v0.ipynb


In [2]:
class name_searcher():
    
    def __init__(self, analyser):
        
        self.analyser = analyser
        
    def assign_name(self, original_name, new_name, dictionary, iteration):
        if new_name in dictionary:
            iteration += 1
            return self.assign_name(original_name, original_name + " " + str(iteration), dictionary, iteration)
        return new_name

    def rank_named_addresses(self, address_score, centralityType, num_top_addresses):

        number_of_top_addresses = num_top_addresses
        address_owners = {}
        url_start = "https://etherscan.io/address/"

        # cleaning address owner data
        clean_owners = {}
        count_unknown = 0

        for index, row in address_score.head(number_of_top_addresses).iterrows():

            address = row["address"]
            if(str(self.analyser.get_node_name(address)[0]) == "None"):

                parser = etherscanParser("https://etherscan.io/address/" + address)
                address_owners[address] = {}
                address_owners[address]['href'] = parser.get_labels_through_href() 
                address_owners[address]['title'] = parser.get_public_address_tag()

                if "Address" not in address_owners[address]['title'] and 'Ethereum Accoun' not in address_owners[address]['title'] and address_owners[address]['href'] != '' and 'Ethereum BlockChain Explorer' not in address_owners[address]['title']:
                    clean_owners[address] = address_owners[address]['title'] + " " + address_owners[address]['href']
                    self.analyser.set_node_name(address,clean_owners[address])
                elif "Address" not in address_owners[address]['title'] and 'Ethereum Accoun' not in address_owners[address]['title'] and 'Ethereum BlockChain Explorer' not in address_owners[address]['title']:
                    clean_owners[address] = address_owners[address]['title']
                    self.analyser.set_node_name(address,clean_owners[address])
                elif address_owners[address]['href'] != '':
                    clean_owners[address] = address_owners[address]['href']
                    self.analyser.set_node_name(address,clean_owners[address])
                else:
                    clean_owners[address] = "Unknown" + str(count_unknown)
                    count_unknown += 1

            else:
                clean_owners[address] = str(self.analyser.get_node_name(address)[0])


        top_scores = {}
        pr_centrality_scores= address_score.set_index('address')[centralityType].to_dict()

        for address in clean_owners.keys():
            name = self.assign_name(clean_owners[address], clean_owners[address], top_scores, 0)
            top_scores[name] = pr_centrality_scores[address]

        top_scorers_df = pd.DataFrame.from_dict(data = top_scores, columns = [centralityType], orient='index')
        #top_scorers_df.plot.bar()

        return top_scorers_df, clean_owners